In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 7.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install tweepy
!pip install ibm_watson
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     |████████████████████████████████| 381 kB 8.3 MB/s 
     |████████████████████████████████| 198 kB 13.2 MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.0.0-py3-none-any.whl size=375160 sha256=3764b47ea7b6fcfce1f05ad2a45886332124a1c7926f617d0fa3194b0a9913a7
  Stored in directory: /root/.cache/pip/wheels/7a/52/a2/540b1f7f1c79d6ec90f9807c3bb1d9f9ff8e1b486c7331256c
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.7.3-py3-none-any.whl size=45941 sha256=5935ed52aa468227f5b1df6ff857d582adf2420bf33c55fdf4324cdf91e02d82
  Stored in directory: /root/.cache/pip/wheels/42/3b/dc/7245cd35096169b10a629ca9cec6d42d684886f97db724bc76
Successfully built ibm-watson ibm-cloud-sdk-core


In [ ]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
#import preprocessor as p
import ibm_watson

In [ ]:
consumer_key = 'AdVCM5CnJ00NJ8jPSXrTVmA39'
consumer_secret = 'V7nfJ6eTFERY4vXsfpDDz6f75bFPEncNisWDkcSOqsX6hZRLIl'
access_key = '1252209964199710721-hmcuVrta5CYXtLIgRZvN2ktY7Fa1cc'
access_secret = 'mh7LzyOukZ2moG47dbTkC9F6ZVN6Mm4A6cPNW6PJEYKyP'


In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


In [ ]:
results=[]
for tweet in tweepy.Cursor(api.search, q = '#lockdown',since="2020-03-22", lang = "en").items(200):
    results.append(tweet)

In [ ]:
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])

    data_set["text"] = [tweet.text for tweet in results]
    return data_set
dataset=tweets_df(results)
print(dataset)

                      id                                               text
0    1340208371102867457  RT @BitSigmaBTC: 🦸OKEx Hero Recruitment\n\nIf ...
1    1340208339905466369  RT @Drvilasjagdale: https://t.co/rQG7Q3i3kj my...
2    1340208275300769792  RT @paschal2k2: Love is a beautiful thing 😍😍😍😍...
3    1340208273295863808  RT @mi6rogue: In a "Secret" meeting with @Bori...
4    1340208226659426305  Is this inept government going to lock down la...
..                   ...                                                ...
195  1340200791647395840  RT @paschal2k2: Love is a beautiful thing 😍😍😍😍...
196  1340200774224261123  COVID-19 Update For December 18 2020 In Nigeri...
197  1340200689964879874  Shatta wale want to go to Las Vegas 🤣🤣 @shatta...
198  1340200669786107904  RT @paschal2k2: Love is a beautiful thing 😍😍😍😍...
199  1340200589985255425  RT @MennoPP: Absolutely loving using @get_mibo...

[200 rows x 2 columns]


In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
from ibm_watson import ToneAnalyzerV3
import pandas as pd
import time
authenticator = IAMAuthenticator('3RiQuDmUN0q6aGqfv-Ci8-7-OFnk9X-yI8aJ87R3Wz0M')
service = ToneAnalyzerV3(version='2020-09-21',authenticator=authenticator)
service.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/134d7e8c-4b1b-4448-ab93-099178450abc')

In [ ]:
sentiments=pd.DataFrame()
sentiments["id"]=np.nan
sentiments["Sadness"]=np.nan
sentiments["Confident"]=np.nan
sentiments["Tentative"]=np.nan
sentiments["Joy"]=np.nan
sentiments["Anger"]=np.nan
sentiments["Analytical"]=np.nan
t=[]
for index, review in dataset['text'].iteritems():

    json_output = service.tone(tone_input=review)
    for i in json_output.result['document_tone']["tones"]:
            print(i)
            #Append the attributes to the data
            t.append(index)
            sentiments[i['tone_name']] =[i['score']]

#print(t)

{'score': 0.693557, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.914696, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.674901, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.579367, 'tone_id': 'analytical', 'tone_name': 'Analytical'}
{'score': 0.71468, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.698873, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.908592, 'tone_id': 'analytical', 'tone_name': 'Analytical'}
{'score': 0.687249, 'tone_id': 'sadness', 'tone_name': 'Sadness'}
{'score': 0.914696, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.698873, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.908592, 'tone_id': 'analytical', 'tone_name': 'Analytical'}
{'score': 0.914696, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.914696, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.698873, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score': 0.908592, 'tone_id': 'analytical', 'tone_name': 'Analytical'}
{'score': 0.914696, 'tone_id': 'joy', 'tone_name': 'Joy'}
{'score':